In [24]:
import rasterio
import numpy as np
from rasterio.plot import show
from rasterio.crs import CRS
from rasterio import Affine
from datetime import date
from rasterio.warp import reproject
from rasterio.enums import Resampling
from affine import Affine as newAffine
from rasterio.vrt import WarpedVRT

### Create some integrated alert test data

In [ ]:
low = (np.ma.sum([np.ma.masked_equal(glad_lan, 0) & 1, np.ma.masked_equal(glad_s2, 0) & 1, np.ma.masked_equal(radd, 0) & 1], axis=0) == 1) * 1

high = (np.ma.sum([np.ma.masked_equal(glad_lan, 0) & 1, np.ma.masked_equal(glad_s2, 0) & 1, np.ma.masked_equal(radd, 0) & 1], axis=0) == 0) * 2


highest = ((np.ma.sum([(
        np.ma.masked_equal(glad_lan, 0) > 0) * 1,
        (np.ma.masked_equal(glad_s2, 0) > 0) * 1 ,
        (np.ma.masked_equal(radd, 0) > 0) * 1],
    axis=0) >= 2) * 3).filled(0)


highest_and_high = np.ma.sum([
    (np.ma.sum([
        (np.ma.masked_equal(glad_lan, 0) > 0) * 1,
        (np.ma.masked_equal(glad_s2, 0) > 0) * 1 ,
        (np.ma.masked_equal(radd, 0) > 0) * 1],
        axis=0) >= 2) *1,
    ((np.ma.sum([
        np.ma.masked_equal(glad_lan, 0) & 1,
        np.ma.masked_equal(glad_s2, 0) & 1,
        np.ma.masked_equal(radd, 0) & 1
    ], axis=0) == 0) * 2)], axis=0)
    

all_conf = np.ma.sum([
    np.clip(np.ma.sum([
        (np.ma.sum([
            (np.ma.masked_equal(glad_lan, 0) > 0) * 1,
            (np.ma.masked_equal(glad_s2, 0) > 0) * 1 ,
            (np.ma.masked_equal(radd, 0) > 0) * 1],
            axis=0) >= 2) * 2,
        (np.ma.sum([
            np.ma.masked_equal(glad_lan, 0) & 1,
            np.ma.masked_equal(glad_s2, 0) & 1,
            np.ma.masked_equal(radd, 0) & 1
        ], axis=0) == 0) * 2,
        (np.ma.sum([
            np.ma.masked_equal(glad_lan, 0) & 1,
            np.ma.masked_equal(glad_s2, 0) & 1,
            np.ma.masked_equal(radd, 0) & 1],
            axis=0) == 1) * 1          
        ], axis=0), None, 3).filled(0) * 10000, 
    np.ma.min([
            np.ma.masked_equal(glad_lan, 0) >> 1,
            np.ma.masked_equal(glad_s2, 0) >> 1,
            np.ma.masked_equal(radd, 0) >> 1],
        axis=0)
], axis=0).filled(0)
        


all_conf

### Save result to file 

In [ ]:
meta = {
    'driver': 'GTiff',
     'dtype': 'uint16',
     'nodata': 0.0,
     'width': 10,
     'height': 10,
     'count': 1,
     'crs': CRS.from_epsg(4326),
     'transform': Affine(0.0001, 0.0, 40.0,
            0.0, -0.0001, -20.0)}

with rasterio.open('integrated_confidence.tif', 'w', **meta) as int_file:
    int_file.write(all_conf, 1)

### Verify data saved correctly

In [ ]:
with rasterio.open('integrated_confidence.tif') as int_file:
    conf_data = int_file.read(1)

### Calc function to get the earliest alert 

In [ ]:
earliest_alert_date = np.ma.min([
        np.ma.masked_equal(glad_lan, 0) >> 1,
        np.ma.masked_equal(glad_s2, 0) >> 1,
        np.ma.masked_equal(radd, 0) >> 1],
    axis=0).filled(0)

earliest_alert_date

### Save alert date to file

In [ ]:
with rasterio.open('earliest_alert.tif', 'w', **meta) as alert_file:
    alert_file.write(earliest_alert_date, 1)

In [5]:
res_file_16 = rasterio.open('/Users/solomon.negusse/wri/resampled_2048th_min.tif')
glad_16 = res_file_16.read(1, masked=True)
    
with rasterio.open('/Users/solomon.negusse/wri/resampled_4096th_min.tif') as res_file_8:
    glad_8 = res_file_8.read(1, masked=True)

In [ ]:
print(glad_8)

In [12]:
print(res_file_16.profile)
print(res_file_16.meta)

{'driver': 'GTiff', 'dtype': 'uint16', 'nodata': 0.0, 'width': 16, 'height': 16, 'count': 3, 'crs': CRS.from_epsg(3857), 'transform': Affine(39135.75848225398, 0.0, -2504688.542848654,
       0.0, -39135.75848225398, 3757032.814319771), 'blockxsize': 16, 'blockysize': 16, 'tiled': True, 'interleave': 'pixel'}
{'driver': 'GTiff', 'dtype': 'uint16', 'nodata': 0.0, 'width': 16, 'height': 16, 'count': 3, 'crs': CRS.from_epsg(3857), 'transform': Affine(39135.75848225398, 0.0, -2504688.542848654,
       0.0, -39135.75848225398, 3757032.814319771)}


In [22]:
# with rasterio.open('resample_rasterio.tif', 'w', **res_file_16.profile, height) as dst:
# for i, band in enumerate(glad_16, 1):
dest = np.zeros((8, 8))
dest_transform = res_file_16.transform * newAffine.scale(0.5)
reproject(
    glad_16,
    dest,
    src_transform=res_file_16.transform,
    src_crs=res_file_16.crs,
#     dst_transform=dest_transform,
    dst_crs=res_file_16.crs,
    resampling=Resampling.min, src_nodata=0)

#         dst.write(dest, indexes=i)

(array([[   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
        [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
        [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
        [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
        [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
        [   0.,    0.,    0.,    0.,    0.,    0., 2202.,    0.],
        [   0.,    0.,    0.,    0.,    0.,    0., 2330., 2202.],
        [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.]]),
 Affine(78271.51696450796, 0.0, -2504688.542848654,
        0.0, -78271.51696450796, 3757032.814319771))

In [36]:
with rasterio.open("/Users/solomon.negusse/wri/resampled_2048th_min.vrt", "r") as src:
    with WarpedVRT(src, resampling=Resampling.nearest, **src.profile) as vrt:
        glad_vrt_16 = vrt.read(1, out_shape=(8,8), masked=True)

    
    


In [37]:
print(glad_vrt_16)

[[-- -- -- -- -- -- -- --]
 [-- -- -- -- -- -- -- --]
 [-- -- -- -- -- -- -- --]
 [-- -- -- -- -- -- -- --]
 [-- -- -- -- -- -- -- --]
 [-- -- -- -- -- -- 2202 --]
 [-- -- -- -- -- -- 3258 --]
 [-- -- -- -- -- -- -- --]]


In [47]:
with rasterio.open("/Users/solomon.negusse/wri/resampled_2048th_min.vrt", "r") as src:
    with WarpedVRT(src, resampling=Resampling.min, **src.profile) as vrt:
        glad_vrt_16_min = vrt.read(1, out_shape=(8,8))
        
    dst_profile = src.profile
    dst_profile['driver'] = 'GTiff'
    dst_profile['count'] = 3
    dst_profile["compress"] = "lzw"
    dst_profile["height"] = 8
    dst_profile["width"] = 8
    with rasterio.open("resampled_vrt.tif", "w", **dst_profile) as dest:
        dest.write(glad_vrt_16_min, indexes=1)

In [50]:
with rasterio.open("/Users/solomon.negusse/wri/resampled_2048th_min.vrt", "r") as src:
    src.profile.pop("transform")
    with WarpedVRT(src, resampling=Resampling.min, **src.profile) as vrt:
        glad_vrt_16_min = vrt.read(1, out_shape=(8,8))

array([[   0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0, 2202,    0],
       [   0,    0,    0,    0,    0,    0, 3258,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0]], dtype=uint16)